In [ ]:
!pip install geopy

In [3]:
import urllib.request, json, csv, geopy.distance, os, gc
import pandas as pd
import numpy as np
from datetime import datetime
from time import sleep, strptime, mktime
apikey = ''

In [4]:
GPS_resource_id = 'f2e5503e-927d-4ad3-9500-4ab9e55deb59'
resorce_type = ('1', '2')
GPS_columns = ['Lines', 'VehicleNumber', 'Brigade', 'Time', 'Lon', 'Lat',\
               'res_type', 'qTime', 'd_time', 'd_dist', 'coord', 'uniqID', 'vehID', 'current']
last_GPS_df = pd.DataFrame([], columns=GPS_columns)
csv_name = 'dataGPS'
csv_nr = 1

def delta_time(vehID, new_time):
    last_time = last_GPS_df[last_GPS_df.vehID == vehID]['Time']
    if (len(last_time)>0):
        try:
            last_time = strptime(last_time.values[0], "%Y-%m-%d %H:%M:%S")
            new_time = strptime(new_time, "%Y-%m-%d %H:%M:%S")
            return int(mktime(new_time) - mktime(last_time))
        except:
            print(last_time, new_time, vehID) 
    return -1

def delta_dist(vehID, new_coord):
    last_coord = last_GPS_df[last_GPS_df.vehID == vehID]['coord']
    if (len(last_coord)>0):
        try:
            return round(geopy.distance.distance(last_coord, new_coord).kilometers, 5)
        except:
            print(last_coord, new_coord, vehID)
    return -1

def curr_time(last_time, new_time, scope):
    try:
        last_time = strptime(last_time, "%Y-%m-%d %H:%M:%S")
        #new_time = strptime(new_time, "%Y-%m-%d %H:%M:%S")
        if(last_time>new_time.timetuple()):
            raise Exception('error: last_time > new_time')
        return int(mktime(new_time.timetuple()) - mktime(last_time)) < scope #86400 = 1 day
    except:
        print("error ", last_time, new_time) 
    return True

In [5]:
while True:
    sleep(5)
    gc.collect()
    new_GPS_df = pd.DataFrame([], columns=GPS_columns)
    for one in resorce_type:
        api = f'https://api.um.warszawa.pl/api/action/busestrams_get/?apikey={apikey}&type={one}&resource_id={GPS_resource_id}'
        try:
            with urllib.request.urlopen(api) as url:
                dataGPS = json.loads(url.read().decode())['result']
        except:
            print('url error')
            continue
        if(type(dataGPS)==str): #Błędna metoda lub parametry wywołania
            continue
        df = pd.DataFrame(dataGPS, columns=GPS_columns)      
        df['coord'] = df[ ['Lon', 'Lat'] ].apply(lambda x: \
                (pd.to_numeric(x['Lat'], errors='coerce'), pd.to_numeric(x['Lon'], errors='coerce')), axis=1)        
        df['res_type'] = one
        df['qTime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        df['uniqID'] = df[ ['Lines', 'VehicleNumber', 'Brigade', 'Time'] ]\
            .apply(lambda x: (x['Lines']+'_'+x['VehicleNumber']+'_'+x['Brigade']+'_'+x['Time']), axis=1)
        df['vehID'] = df[ ['Lines', 'VehicleNumber', 'Brigade'] ]\
            .apply(lambda x: (x['Lines']+'_'+x['VehicleNumber']+'_'+x['Brigade']), axis=1)
        new_GPS_df = pd.concat([new_GPS_df, df])
    uniq_items = new_GPS_df[~new_GPS_df.uniqID.isin(last_GPS_df.uniqID)]
    if(len(uniq_items)==0):
        continue
    uniq_items['d_time'] = uniq_items.loc[:, ['vehID', 'Time'] ].apply(lambda x: delta_time(x['vehID'], x['Time']), axis=1)
    uniq_items['d_dist'] = uniq_items.loc[:, ['vehID', 'coord'] ].apply(lambda x: delta_dist(x['vehID'], x['coord']), axis=1)
    #uniq_items['d_time'] = uniq_items[ ['vehID', 'Time'] ].apply(lambda x: delta_time(x['vehID'], x['Time']), axis=1)
    #uniq_items['d_dist'] = uniq_items[ ['vehID', 'coord'] ].apply(lambda x: delta_dist(x['vehID'], x['coord']), axis=1)
    #uniq_items['mean_veloc'] =
    last_GPS_df = pd.concat([last_GPS_df, new_GPS_df])
    last_GPS_df.drop_duplicates(subset=['vehID'], inplace=True, keep='last')
    last_GPS_df['current'] = last_GPS_df[ ['qTime'] ].apply(lambda x: curr_time(x['qTime'], datetime.now(), 86400), axis=1)
    last_GPS_df = last_GPS_df[last_GPS_df.current].copy()
    try:
        with open(csv_name+str(csv_nr)+'.csv', 'a') as f:
            size = os.fstat(f.fileno()).st_size
            print(uniq_items.shape, last_GPS_df.shape, new_GPS_df.shape, size)
            if(size>100000000): csv_nr += 1
        with open(csv_name+str(csv_nr)+'.csv', 'a') as f:
            uniq_items.iloc[:,0:10].to_csv(f, header=False, index=False, sep=',')        
    except:
        print('file error')

(792, 14) (792, 14) (792, 14) 0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(255, 14) (792, 14) (791, 14) 65195
(13, 14) (792, 14) (791, 14) 87272
(239, 14) (793, 14) (792, 14) 88374
(258, 14) (793, 14) (793, 14) 109053
(232, 14) (793, 14) (791, 14) 131377
(248, 14) (793, 14) (792, 14) 151427
(250, 14) (794, 14) (792, 14) 172886
(13, 14) (794, 14) (792, 14) 194541
(237, 14) (794, 14) (790, 14) 195639
(252, 14) (794, 14) (791, 14) 216172
(244, 14) (794, 14) (791, 14) 237941
(263, 14) (794, 14) (792, 14) 259025
(229, 14) (794, 14) (791, 14) 281799
(254, 14) (794, 14) (791, 14) 301608
(261, 14) (794, 14) (790, 14) 323602
(244, 14) (794, 14) (789, 14) 346179
(220, 14) (794, 14) (790, 14) 367301
(240, 14) (795, 14) (792, 14) 386332
(13, 14) (795, 14) (792, 14) 407117
(240, 14) (795, 14) (790, 14) 408207
(236, 14) (795, 14) (790, 14) 428964
(235, 14) (795, 14) (790, 14) 449407
(13, 14) (795, 14) (19, 14) 469718
(264, 14) (795, 14) (791, 14) 470813
(240, 14) (795, 14) (789, 14) 493713
(13, 14) (795, 14) (789, 14) 514491
(225, 14) (795, 14) (788, 14) 515580
(237, 14) 

(160, 14) (809, 14) (739, 14) 3638876
(154, 14) (809, 14) (740, 14) 3652796
(153, 14) (809, 14) (739, 14) 3666189
(158, 14) (809, 14) (740, 14) 3679512
(160, 14) (809, 14) (739, 14) 3693275
(148, 14) (809, 14) (739, 14) 3707189
(164, 14) (809, 14) (740, 14) 3720032
(153, 14) (809, 14) (739, 14) 3734322
(157, 14) (809, 14) (740, 14) 3747605
(153, 14) (809, 14) (737, 14) 3761270
(156, 14) (809, 14) (739, 14) 3774584
(152, 14) (809, 14) (737, 14) 3788161
(154, 14) (809, 14) (738, 14) 3801377
(152, 14) (809, 14) (737, 14) 3814793
(152, 14) (809, 14) (736, 14) 3827995
(150, 14) (809, 14) (736, 14) 3841221
(149, 14) (809, 14) (735, 14) 3854258
(148, 14) (809, 14) (736, 14) 3867210
(145, 14) (809, 14) (735, 14) 3880099
(152, 14) (809, 14) (736, 14) 3892741
(147, 14) (809, 14) (735, 14) 3905996
(151, 14) (809, 14) (736, 14) 3918796
(151, 14) (809, 14) (735, 14) 3931963
(147, 14) (809, 14) (736, 14) 3945110
(144, 14) (809, 14) (735, 14) 3957931
(150, 14) (809, 14) (735, 14) 3970492
(150, 14) (8

(118, 14) (817, 14) (707, 14) 5987670
(123, 14) (817, 14) (709, 14) 5997970
(116, 14) (817, 14) (708, 14) 6008700
(119, 14) (817, 14) (709, 14) 6018813
(119, 14) (817, 14) (708, 14) 6029183
(119, 14) (817, 14) (709, 14) 6039566
(116, 14) (817, 14) (707, 14) 6049937
(119, 14) (817, 14) (707, 14) 6060038
(118, 14) (817, 14) (708, 14) 6070422
(117, 14) (817, 14) (707, 14) 6080721
(119, 14) (817, 14) (706, 14) 6090937
(114, 14) (817, 14) (705, 14) 6101324
(121, 14) (817, 14) (706, 14) 6111253
(118, 14) (817, 14) (705, 14) 6121789
(112, 14) (817, 14) (706, 14) 6132079
(119, 14) (817, 14) (705, 14) 6141830
(117, 14) (817, 14) (705, 14) 6152199
(123, 14) (817, 14) (706, 14) 6162414
(115, 14) (817, 14) (705, 14) 6173144
(114, 14) (817, 14) (706, 14) 6183162
(117, 14) (817, 14) (705, 14) 6193118
(102, 14) (817, 14) (706, 14) 6203304
(114, 14) (817, 14) (704, 14) 6212144
(122, 14) (817, 14) (706, 14) 6222079
(118, 14) (817, 14) (705, 14) 6232733
(118, 14) (817, 14) (705, 14) 6243010
(116, 14) (8

...

(979, 14) (4526, 14) (1623, 14) 42735602
(1032, 14) (4526, 14) (1622, 14) 42819593
(994, 14) (4525, 14) (1625, 14) 42908229
(1021, 14) (4525, 14) (1627, 14) 42993533
(999, 14) (4526, 14) (1623, 14) 43081136
(251, 14) (4526, 14) (1623, 14) 43166801
(978, 14) (4524, 14) (1626, 14) 43188097
(876, 14) (4525, 14) (1625, 14) 43271901
(960, 14) (4526, 14) (1624, 14) 43346769
(615, 14) (4526, 14) (1625, 14) 43429151
(856, 14) (4526, 14) (1621, 14) 43481557
(960, 14) (4526, 14) (1621, 14) 43554966
(885, 14) (4525, 14) (1622, 14) 43637464
(1018, 14) (4526, 14) (1627, 14) 43713222
(982, 14) (4525, 14) (1628, 14) 43800762
(1030, 14) (4525, 14) (1626, 14) 43884807
(970, 14) (4526, 14) (1627, 14) 43973355
(980, 14) (4526, 14) (1626, 14) 44056656
(999, 14) (4524, 14) (1629, 14) 44140791
(1015, 14) (4524, 14) (1630, 14) 44226633
(1056, 14) (4524, 14) (1629, 14) 44313705
(974, 14) (4525, 14) (1630, 14) 44404374
(952, 14) (4525, 14) (1633, 14) 44487974
(970, 14) (4526, 14) (1630, 14) 44569795
(1109, 14)

(1150, 14) (4613, 14) (1736, 14) 59218993
(1021, 14) (4614, 14) (1735, 14) 59318025
(1103, 14) (4614, 14) (1737, 14) 59405744
(1130, 14) (4615, 14) (1735, 14) 59500604
(1022, 14) (4615, 14) (1737, 14) 59597904
(1081, 14) (4616, 14) (1738, 14) 59685788
(1086, 14) (4617, 14) (1740, 14) 59778915
(1064, 14) (4617, 14) (1739, 14) 59872480
(1104, 14) (4617, 14) (1741, 14) 59964109
(1101, 14) (4616, 14) (1739, 14) 60059202
(1098, 14) (4615, 14) (1739, 14) 60153895
(1105, 14) (4616, 14) (1742, 14) 60248384
(1133, 14) (4616, 14) (1740, 14) 60343545
(1120, 14) (4617, 14) (1740, 14) 60440826
(1172, 14) (4618, 14) (1741, 14) 60537428
(1023, 14) (4619, 14) (1743, 14) 60638545
(933, 14) (4619, 14) (1741, 14) 60726476
(292, 14) (4619, 14) (1741, 14) 60806672
(804, 14) (4621, 14) (1745, 14) 60831557
(1056, 14) (4622, 14) (1748, 14) 60901083
(1152, 14) (4624, 14) (1751, 14) 60992017
(999, 14) (4625, 14) (1750, 14) 61091410
(1169, 14) (4626, 14) (1752, 14) 61177372
(1159, 14) (4628, 14) (1755, 14) 61278

(331, 14) (4706, 14) (345, 14) 77251189
(1084, 14) (4706, 14) (1873, 14) 77279335
(1134, 14) (4709, 14) (1874, 14) 77373436
(1115, 14) (4709, 14) (1875, 14) 77471148
(1131, 14) (4709, 14) (1877, 14) 77567161
(1146, 14) (4711, 14) (1880, 14) 77664694
(1152, 14) (4711, 14) (1878, 14) 77763427
(335, 14) (4711, 14) (349, 14) 77862788
(1378, 14) (4711, 14) (1882, 14) 77891374
(1156, 14) (4710, 14) (1882, 14) 78010539
(1133, 14) (4710, 14) (1881, 14) 78110150
(1143, 14) (4711, 14) (1880, 14) 78207785
(1146, 14) (4711, 14) (1857, 14) 78306329
(1210, 14) (4712, 14) (1881, 14) 78405177
(1074, 14) (4713, 14) (1881, 14) 78509539
(1109, 14) (4714, 14) (1881, 14) 78602000
(1142, 14) (4714, 14) (1881, 14) 78697551
(1124, 14) (4714, 14) (1884, 14) 78796023
(1147, 14) (4715, 14) (1887, 14) 78893010
(1164, 14) (4715, 14) (1886, 14) 78992038
(1287, 14) (4717, 14) (1886, 14) 79092362
(1353, 14) (4719, 14) (1890, 14) 79203635
(1223, 14) (4721, 14) (1894, 14) 79320734
(1310, 14) (4721, 14) (1896, 14) 79426

(1179, 14) (4765, 14) (1964, 14) 95345182
(1291, 14) (4765, 14) (1963, 14) 95446930
(1404, 14) (4765, 14) (1963, 14) 95558616
(1254, 14) (4765, 14) (1961, 14) 95680034
(1387, 14) (4765, 14) (1960, 14) 95787925
(1289, 14) (4765, 14) (1960, 14) 95907998
(1239, 14) (4765, 14) (1961, 14) 96019380
(1366, 14) (4765, 14) (1957, 14) 96126288
(1358, 14) (4766, 14) (1960, 14) 96244436
(1264, 14) (4766, 14) (1961, 14) 96361901
(1316, 14) (4767, 14) (1960, 14) 96471059
(1360, 14) (4767, 14) (1962, 14) 96584586
(1297, 14) (4767, 14) (1963, 14) 96702325
(1291, 14) (4770, 14) (1963, 14) 96814396
(1043, 14) (4771, 14) (1963, 14) 96926051
(1271, 14) (4771, 14) (1964, 14) 97016227
(1266, 14) (4771, 14) (1967, 14) 97126218
(1277, 14) (4773, 14) (1969, 14) 97235603
(363, 14) (4773, 14) (383, 14) 97345932
(1498, 14) (4773, 14) (1964, 14) 97376997
(1239, 14) (4774, 14) (1969, 14) 97506885
(1262, 14) (4775, 14) (1961, 14) 97613930
(1270, 14) (4775, 14) (1965, 14) 97722989
(1405, 14) (4775, 14) (1968, 14) 978

KeyboardInterrupt: 